# Reconocimiento de imágenes con TensorFlow. REDES NEURONALES CONVOLUCIONALES (CNN)

In [1]:
#!pip install --upgrade pip
#!pip install pandas 
#!pip install tqdm 
#!pip install scikit-learn 
#!pip install scikit-image
#!pip install imblearn
#!pip install import-ipynb

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import os
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm #barra de progreso
import cv2

#para redimensionar
import sklearn
import skimage
from skimage.transform import resize

import random

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from skimage.color import rgb2gray

#para cargar imágenes desde internet usaremos estas librerías
from PIL import Image
import requests
from io import BytesIO

## Funciones

In [2]:
# Preprocesado: redimensionado 150x150, escala de grises (eliminar capa de color) y conversión a array
def preprocess(image):
    image = skimage.transform.resize(image, (150,150,3), mode = 'constant', anti_aliasing=True)
    image = rgb2gray(image)
    image_array = np.asarray(image)
    return image_array

In [3]:
# Carga de datos
def get_images(dir): #dir = chest_xray/train/ o chest_xray/test/
    IMAGES = []
    LABELS = []
    
    for subdir in os.listdir(dir): #lista de subdirectorios en train/ o test/ -> ['NORMAL', 'PNEUMONIA']
        if not subdir.startswith('.'):
            if subdir in ['NORMAL']:
                label = 0
            elif subdir in ['PNEUMONIA']:
                label = 1
            else: 
                label = 2
                    
            for image_name in tqdm(os.listdir(dir+subdir)): #barra de progreso
                image = cv2.imread(dir + subdir + '/' + image_name) #lectura
                if image is not None:
                    #bloque de preprocesado
                    image_array = preprocess(image)    
                                 
                    #add a listas de arrays y de labels
                    IMAGES.append(image_array) 
                    LABELS.append(label) 
        
    IMAGES = np.asarray(IMAGES)
    LABELS = np.asarray(LABELS)
    return IMAGES, LABELS

In [4]:
LOADED = True
if LOADED == False:
    x_train, y_train = get_images("chest_xray/train/") #nº de imgs normal = nº imgs pneu
    x_test, y_test = get_images("chest_xray/test/") #no dimensionados
    
    #almacenamiento de arrays en archivos
    np.save('xtrain.npy', x_train)
    np.save('ytrain.npy', y_train)
    np.save('xtest.npy', x_test)
    np.save('ytest.npy', y_test)            
    
else:
    x_train = np.load('xtrain.npy')
    y_train = np.load('ytrain.npy')
    x_test = np.load('xtest.npy')
    y_test = np.load('ytest.npy')

100%|██████████| 390/390 [01:15<00:00,  5.17it/s]


In [5]:
x_train = x_train.reshape(len(x_train),150,150,1)
x_test = x_test.reshape(len(x_test),150,150,1)

## Definición del modelo de CNN

In [6]:
model = models.Sequential()

# 1) Se crea un mapa o array de valores más pequeño que caracteriza a la imagen
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 1)))

# 2) Se aplica un filtro MAX (obtiene el valor máximo de cada sección de píxeles) para obtener una capa de reducción o imagen más pequeña
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())


model.summary() #resumen de la estructura de la red neuronal
#los parámetros son los pesos

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 64)       0